<a href="https://colab.research.google.com/github/spirosChv/smartNetsWorkshop/blob/main/neuron/practical3_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulating dendrites - Part 1: How inputs propagate to the soma and how inputs interact.

In this notebook we will see how inputs attenuate towards the soma. Also we will see how inputs integrate in a dendritic branch. 

In [ ]:
!pip install neuron --quiet

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from neuron import h
h.load_file("stdrun.hoc")

In [ ]:
# @title Make nicer plots -- Execute this cell
def mystyle():
  """
  Create custom plotting style.

  Returns
  -------
  my_style : dict
      Dictionary with matplotlib parameters.

  """
  # color pallette
  style = {
      # Use LaTeX to write all text
      "text.usetex": False,
      "font.family": "DejaVu Sans",
      "font.weight": "bold",
      # Use 16pt font in plots, to match 16pt font in document
      "axes.labelsize": 16,
      "axes.titlesize": 20,
      "font.size": 16,
      # Make the legend/label fonts a little smaller
      "legend.fontsize": 14,
      "xtick.labelsize": 14,
      "ytick.labelsize": 14,
      "axes.linewidth": 2.5,
      "lines.markersize": 10.0,
      "lines.linewidth": 2.5,
      "xtick.major.width": 2.2,
      "ytick.major.width": 2.2,
      "axes.labelweight": "bold",
      "axes.spines.right": False,
      "axes.spines.top": False
  }

  return style


plt.style.use("seaborn-colorblind")
plt.rcParams.update(mystyle())

Define the simulation parameters

In [ ]:
# Simulation parameters	
tstop = 100  # simulation time (ms)
h.dt = 0.1  # integration step (ms)
vinit = -65  # initial voltage (mV)

### Create a soma and two dendrites, define their anatomical and biophysical properties and connect them.

In [ ]:
soma = h.Section(name='soma')
dend0, dend1 = [h.Section(name=n) for n in ['dend0', 'dend1']]

# Define properties of soma
soma.diam = 20  # um
soma.L = 200  # um
soma.Ra = 100  # Axial resistance in Ohm * cm
soma.cm = 1  # specific membrane capacitance, capacitance per unit of membrane, uF/cm^2   
soma.nseg = 1  # number of segments (actual compartments)	

# Insert the hh channels (inluding leak)
soma.insert('hh')
for seg in soma: 
  seg.hh.gnabar = 0.12  # Sodium conductance in S/cm2
  seg.hh.gkbar = 0.036  # Potassium conductance in S/cm2
  seg.hh.gl = 0.0003  # Leak conductance in S/cm2
  seg.hh.el = -65  # Reversal potential in mV

# Define properties of dend0
dend0.diam = 5
dend0.L = 500
dend0.nseg = 11
dend0.cm = 1
dend0.Ra = 100

# Insert passive (leak) channels
dend0.insert('pas')      
for seg in dend0:
  seg.pas.e = -65  # leak reversal potential in mV
  seg.pas.g = 0.0003  # leak maximal conductance in S/cm2
	
# Define properties of dend1
dend1.diam = 3
dend1.L = 200
dend1.nseg = 13
dend1.cm = 1
dend1.Ra = 100

# Insert passive (leak) channels
dend1.insert('pas')     
for seg in dend0:
  seg.pas.e = -65  # leak reversal potential in mV
  seg.pas.g = 0.0003  # leak maximal conductance in S/cm2

# Connect the `0` point of `dend0` to the `0` point of the soma, i.e., soma(0)
dend0.connect(soma(0), 0)
# Connect the `1` point of `dend0`, i.e., dend0(1), to the `0` point of the dend1, i.e., dend1(0)
dend1.connect(dend0(1), 0)

### Synaptic Stimulation

In [ ]:
# Place the synapse to the middle (0.5) of the soma...
syn = h.ExpSyn(soma(0.5))
syn.e = 0  # Reverse potential of the synapse (mV)

#...create an artificial spike (an "event" to be delivered to the synapse)...
tsignal = 50  # presynaptic spike time
ns = h.NetStim(0.5)  # crete the presynaptic spike
ns.start = tsignal  # set the presynaptic start
ns.number = 1  # set the numbers of spikes

#... and connect the event to the synapse.
nc = h.NetCon(ns, syn)  # create the NetCon object
nc.delay = 0  # set the delay (ms)
nc.weight[0] = 0.008*1  # set the weight

### Example simulation
### Calculate and print in terminal the depolarization at the soma and at the middle part of the distal dendrite (`dend1`)

In [ ]:
vsoma_vec = h.Vector().record(soma(0.5)._ref_v)  # Membrane potential vector
vdend1_vec = h.Vector().record(dend1(0.5)._ref_v)  # Membrane potential vector
t_vec = h.Vector().record(h._ref_t)  # Time stamp vector

# Run the simulation
h.finitialize(vinit)
h.continuerun(tstop)

# Remove the first 20ms to avoid artifacts
tremove = 20
vsoma_vec.remove(0, int(tremove/h.dt))
vdend1_vec.remove(0, int(tremove/h.dt))
t_vec.remove(0, int(tremove/h.dt))

plt.figure(figsize=(8, 6))
plt.plot(t_vec, vsoma_vec, color='black', label='soma')
plt.plot(t_vec, vdend1_vec, color='red', label='dend')
plt.xlabel('time (ms)')
plt.ylabel('mV')
plt.legend()
plt.show()

print (f'\nSomatic depolarization is {np.round(vsoma_vec.max()-vsoma_vec[int((tsignal-tremove)/h.dt)-1], 2)} mV')
print (f'Distal Dendritic depolarization is {np.round(vdend1_vec.max()-vdend1_vec[int((tsignal-tremove)/h.dt)-1], 2)} mV')

### Task 1: How can you easily make this synapse inhibitory?

<details>
<summary>Click to see the answer!</summary>

We have to reduce the reversal potential of the synapse below the resting membrane potential.

```python
syn.e = -70
```

In [ ]:
# @markdown Re-execute the code!

value = -70  # @param {type:"number"}
syn.e = value

print(f"The reversal potetnial of the synapse has been set to {value} mV")

In [ ]:
# Run the simulation
h.finitialize(vinit)
h.continuerun(tstop)

# Remove the first 20ms to avoid artifacts
tremove = 20
vsoma_vec.remove(0, int(tremove/h.dt))
vdend1_vec.remove(0, int(tremove/h.dt))
t_vec.remove(0, int(tremove/h.dt))

plt.figure(figsize=(8, 6))
plt.plot(t_vec, vsoma_vec, color='black', label='soma')
plt.plot(t_vec, vdend1_vec, color='red', label='dend')
plt.xlabel('time (ms)')
plt.ylabel('mV')
plt.legend()
plt.show()

### Task 2: Move the synapse to the middle of `dend1` section.
### What do you expect for the amplitude and the duration of the EPSP seen at the dendrite and at the soma?

<details>
<summary>Ckick here to see the answer</summary>

```python
syn = h.ExpSyn(dend1(0.5))
```

In [ ]:
# @title Re-execute the code
# Place the synapse to the middle (0.5) of the dend1...
syn = h.ExpSyn(dend1(0.5))
syn.e = 0  # Reverse potential of the synapse (mV)

#... and connect the event to the synapse.
nc = h.NetCon(ns, syn)  # create the NetCon object
nc.delay = 0  # set the delay (ms)
nc.weight[0] = 0.008*1  # set the weight

In [ ]:
# Run the simulation
h.finitialize(vinit)
h.continuerun(tstop)

# Remove the first 20ms to avoid artifacts
tremove = 20
vsoma_vec.remove(0, int(tremove/h.dt))
vdend1_vec.remove(0, int(tremove/h.dt))
t_vec.remove(0, int(tremove/h.dt))

plt.figure(figsize=(8, 6))
plt.plot(t_vec, vsoma_vec, color='black', label='soma')
plt.plot(t_vec, vdend1_vec, color='red', label='dend')
plt.xlabel('time (ms)')
plt.ylabel('mV')
plt.legend()
plt.show()

print (f'\nSomatic depolarization is {np.round(vsoma_vec.max()-vsoma_vec[int((tsignal-tremove)/h.dt)-1], 2)} mV')
print (f'Distal Dendritic depolarization is {np.round(vdend1_vec.max()-vdend1_vec[int((tsignal-tremove)/h.dt)-1], 2)} mV')

### Task 3: Insert 1 to 5 synapses like this one in the same location. In this case, this is equivalent to increasing the weight 1 to 5 times.
### What is the expected (arithmetic sum) versus actual (simulated output)?
### Is their relationship linear? 

In [ ]:
nsyn = 11 # @param {type:"slider", min:1, max:25, step:1}
nc.weight[0] = 0.008*nsyn  # set the weight

# Run the simulation
h.finitialize(vinit)
h.continuerun(tstop)

# Remove the first 20ms to avoid artifacts
tremove = 20
vsoma_vec.remove(0, int(tremove/h.dt))
vdend1_vec.remove(0, int(tremove/h.dt))
t_vec.remove(0, int(tremove/h.dt))

plt.figure(figsize=(8, 6))
plt.plot(t_vec, vsoma_vec, color='black', label='soma')
plt.plot(t_vec, vdend1_vec, color='red', label='dend')
plt.xlabel('time (ms)')
plt.ylabel('mV')
plt.legend()
plt.show()

print (f'\nSomatic depolarization is {np.round(vsoma_vec.max()-vsoma_vec[int((tsignal-tremove)/h.dt)-1], 2)} mV')
print (f'Distal Dendritic depolarization is {np.round(vdend1_vec.max()-vdend1_vec[int((tsignal-tremove)/h.dt)-1], 2)} mV')

Now, let's run the loop!

In [ ]:
N = 25
actual_epsp = []
for i in range(N):
  nc.weight[0] = 0.008*(i+1)

  # reinitialize the simulator and run again
  h.finitialize(vinit)
  h.continuerun(tstop)
  
  # Remove the first 20ms to avoid artifacts
  tremove = 20
  vsoma_vec.remove(0, int(tremove/h.dt))
  t_vec.remove(0, int(tremove/h.dt))
  actual_epsp.append(np.round(vsoma_vec.max() - vsoma_vec[int((tsignal-tremove)/h.dt)-1], 2))

# Calculate the expected EPSP
expected_epsp = [actual_epsp[0]*i for i in range(1, N+1)]

# Plots the results
plt.figure(figsize=(8, 6)) 
plt.plot(expected_epsp, actual_epsp, color='red', label='actual')
plt.plot(expected_epsp, expected_epsp, linestyle='dashed', color='black', label='linear')
plt.xlabel('Expected EPSP-Linear Summation (mV)')
plt.ylabel('Actual EPSP (mV)')
plt.legend()
plt.show()